# Set up

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import jax.numpy as jnp
import jax
import time
from torch.utils import data
from jax import grad, jit, random
from tqdm import tqdm
from math import sqrt
from google.colab import files
from itertools import combinations
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, auc, mean_squared_error, r2_score

## Import Data

In [ ]:
# read data
train = pd.read_csv('movie_ratings_train.csv')
test = pd.read_csv('movie_ratings_test.csv')
movies = pd.read_csv('movies.csv')

# add label
train['data_source'] = 'train'
test['data_source'] = 'test'

train.head()
test.head()

,userId,movieId,timestamp,data_source
0,1,349,964982563,test
1,1,592,964982271,test
2,1,780,964984086,test
3,1,1196,964981827,test
4,1,1208,964983250,test


# Data Preprocessing

Because we want to use models like Regression or Neural Network, so our model needs information of movie's genres. What we gonna do next is we will concat our train and test to a whole dataset, then we will apppend all mandatory information to that dataset and split again so that our model can learn information like genres.

In [ ]:
whole_df = pd.concat([train, test], ignore_index=True)
whole_df.head()

,userId,movieId,rating,timestamp,data_source
0,1,1,4.0,964982703,train
1,1,3,4.0,964981247,train
2,1,6,4.0,964982224,train
3,1,47,5.0,964983815,train
4,1,50,5.0,964982931,train


We will use information in 'movies.csv' to help us.

In [ ]:
# Merge the genres into whole_df based on movieId
movies_relevant_df = movies[['movieId', 'genres']]  # Select only relevant columns
whole_df_with_genres = pd.merge(whole_df, movies_relevant_df, on='movieId', how='left')

# Display the first few rows of the updated whole_df
whole_df_with_genres.head()

,userId,movieId,rating,timestamp,data_source,genres
0,1,1,4.0,964982703,train,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,train,Comedy|Romance
2,1,6,4.0,964982224,train,Action|Crime|Thriller
3,1,47,5.0,964983815,train,Mystery|Thriller
4,1,50,5.0,964982931,train,Crime|Mystery|Thriller


Now, we have extract all movie genres from movies.csv file and append to our whole_df dataset. Next, we will use one-hot encode our genres column since our model only accept numerical values and there are only 20 genres.

In [ ]:
# Split the 'genres' column into individual genre tags and extract unique genres
unique_genres = set(
    genre for genres in whole_df_with_genres['genres'].dropna()
    for genre in genres.split('|')
)

# Display all unique genres
print("Unique genres:", unique_genres)
print(len(unique_genres))

Unique genres: {'Horror', 'Adventure', 'Romance', 'Sci-Fi', 'Musical', 'Fantasy', 'Comedy', 'Western', 'Thriller', '(no genres listed)', 'Drama', 'Film-Noir', 'Mystery', 'War', 'Documentary', 'IMAX', 'Children', 'Animation', 'Action', 'Crime'}
20


## Encoding

First, we need to deal with '|' delimiter since get_dummies() treats the entire string as a single category.

In [ ]:
# Split the genres column into dummy variables
genre_split = whole_df_with_genres['genres'].str.get_dummies(sep='|')

# Append these genre dummy columns to the original dataframe
whole_df_with_dummies = pd.concat([whole_df_with_genres, genre_split], axis=1)

# Drop the original genres column
whole_df_with_dummies = whole_df_with_dummies.drop(columns=['genres'])

# Display the first few rows of the updated dataframe
whole_df_with_dummies.head()

,userId,movieId,rating,timestamp,data_source,(no genres listed),Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,964982703,train,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,964981247,train,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,964982224,train,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,964983815,train,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,964982931,train,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


By looking the cleaned dataset, we noticed that there is a column called '(no genres listed)' which means there are some movies that have no genre. From the following chunk, we know there are only 47 rows that ['(no genres listed)'] == 1. Because our original dataset is large enough, so we can just remove these 47 rows to avoid some unnecessary troubles in the furture.

In [ ]:
print(len(whole_df_with_dummies[whole_df_with_dummies['(no genres listed)'] == 1]))

# Drop rows where '(no genres listed)' is 1
whole_df_with_dummies = whole_df_with_dummies[whole_df_with_dummies['(no genres listed)'] == 0]

print(len(whole_df_with_dummies[whole_df_with_dummies['(no genres listed)'] == 1]))

47
0


In the next, we will do some features engineering and compare different model and choose the best one based on their performences. First thing we need to do is split our whole dataset back to training set and testing set.

In [ ]:
whole_df_with_dummies = whole_df_with_dummies.drop(columns=['timestamp'])

# split the whole_df back to train and test data frame
train_df = whole_df_with_dummies[whole_df_with_dummies['data_source'] == 'train'].drop(columns = ['data_source'])
test_df = whole_df_with_dummies[whole_df_with_dummies['data_source'] == 'test'].drop(columns = ['data_source', 'rating'])

In [ ]:
train_df.shape

(90789, 23)

# Feature selections

We both tried Forward Stepwise Selection and Backward Stepwise Selection.

The result is:

Random Forest Model ------------- Validation MSE ------------- Score

All features: ------------------------------------- 0.9916 -------------------- 99

Forward Features(20): -------------------- 0.9920 -------------------- 99

Backward Features(21): ------------------ 0.9928 ------------------- 99

Thus, we will use features selected by Backward Selection for our Random Forest Model.

In [ ]:
X = train_df[train_df.columns[~train_df.columns.isin(['rating'])]]
y = train_df['rating']

# split our train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Backward Selection

In [ ]:
def processSubset(feature_set, X_train, y_train, X_test, y_test):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y_train,X_train[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X_test[list(feature_set)]) - y_test) ** 2).sum()
    RMSE = sqrt(RSS/len(y_test))
    return {'model':regr, 'RSS':RSS, 'RMSE':RMSE}

def backward(predictors, X_train, y_train, X_test, y_test):

    results = []

    # Generate all models by removing one predictor from the current set
    for combo in combinations(predictors, len(predictors) - 1):
        results.append(processSubset(list(combo), X_train, y_train, X_test, y_test))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose the model with the lowest RSS
    best_model = models.loc[models['RSS'].argmin()]

    return best_model

In [ ]:
# initialize a dataframe to store RSS and model object of each model
backward_models = pd.DataFrame(columns=['RSS', 'model'], index = range(1, len(X.columns)))

# store all features into 'predictores' variables
predictors = X.columns

# call backward stepwise selection
while(len(predictors) > 1):
    backward_models.loc[len(predictors) - 1] = backward(predictors, X_train, y_train, X_test, y_test)
    predictors = backward_models.loc[len(predictors) - 1]['model'].model.exog_names

# create a list to store RSS value and errors of each model
RSS_b = []

# store each RSS values
for m in backward_models.RSS:
    RSS_b.append(m)

# find the index of the model with the lowest RSS value
backward_best_features = np.array(RSS_b).argmin()

# find the features and form the best model
best_backward_model = backward_models.loc[backward_models.index[backward_best_features], 'model']

# get list of name of best model
predictors_in_best_model_b = best_backward_model.model.exog_names

# remove intercept/const number
if 'Intercept' in predictors_in_best_model_b:
    predictors_in_best_model_b.remove('Intercept')
elif 'const' in predictors_in_best_model_b:
    predictors_in_best_model_b.remove('const')

# get the numeber of features of the best model
number_of_features = len(predictors_in_best_model_b)

print(f"The number of features of the best model is: {number_of_features}")

The number of features of the best model is: 21


# Models

In [ ]:
X = train_df.drop(columns = ['rating'])
y = train_df['rating']

# split our train data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Ensure y_train is a NumPy array
# y_train = np.array(y_train)
# y_test = np.array(y_test)
y.head()

,rating
0,4.0
1,4.0
2,4.0
3,5.0
4,5.0


## Logistic Regression

In [ ]:
y_train = (y_train * 10).astype(int)
y_test = (y_test * 10).astype(int)
# y_train

In [ ]:
log_reg = LogisticRegressionCV(Cs=10, penalty='l2', random_state = 42)

# Step 3: Train the model
log_reg.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = log_reg.predict(X_test)

# Step 5: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

## Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'max_features': [0.2, 0.4, 0.6, 0.8, 'auto']
# }
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}



model = RandomForestRegressor(random_state=42, n_estimators=500, max_features=0.6,max_depth=10,min_samples_split=5)


# Change n_jobs to a lower value or 1
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=1, verbose=2)
# By setting n_jobs to 1, you can force the GridSearch to run sequentially instead of in parallel.
# This will likely resolve the memory error. If you have more than 1 core, you can gradually increase this
# number and monitor memory usage.


grid_search.fit(X, y)


print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best Score: {grid_search.best_score_}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ................max_depth=None, min_samples_split=2; total time= 2.1min
[CV] END ................max_depth=None, min_samples_split=2; total time= 2.0min
[CV] END ................max_depth=None, min_samples_split=2; total time= 2.0min
[CV] END ................max_depth=None, min_samples_split=2; total time= 2.0min
[CV] END ................max_depth=None, min_samples_split=2; total time= 2.0min
[CV] END ................max_depth=None, min_samples_split=5; total time= 1.8min
[CV] END ................max_depth=None, min_samples_split=5; total time= 1.7min
[CV] END ................max_depth=None, min_samples_split=5; total time= 1.8min
[CV] END ................max_depth=None, min_samples_split=5; total time= 1.8min
[CV] END ................max_depth=None, min_samples_split=5; total time= 1.8min
[CV] END ...............max_depth=None, min_samples_split=10; total time= 1.7min
[CV] END ...............max_depth=None, min_samp

In [ ]:
rf_model = RandomForestRegressor(random_state=42, n_estimators=500, max_features=0.6,max_depth=10,min_samples_split=5)
rf_model.fit(X,y)




RandomForestRegressor(max_depth=10, max_features=0.6, min_samples_split=5,
                      n_estimators=500, random_state=42)

## Neural Network

Code in this part is hevily borrowed from Professor Mihai's lecture and our course github.

In [ ]:
import jax
import jax.numpy as jnp
import optax
from jax import random, grad, jit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

def init_params(layers, key, scale=1e-2):
    params = []
    keys = random.split(key, len(layers) - 1)
    for i in range(len(layers) - 1):
        w_key, b_key = random.split(keys[i])
        weights = random.normal(w_key, (layers[i], layers[i + 1])) * jnp.sqrt(2 / layers[i])
        biases = jnp.zeros(layers[i + 1])
        params.append((weights, biases))
    return params

def forward(params, x):
    for i, (w, b) in enumerate(params[:-1]):
        x = jnp.dot(x, w) + b
        x = jax.nn.tanh(x)  # activation function
    final_w, final_b = params[-1]
    output = jnp.dot(x, final_w) + final_b
    # output = jax.nn.tanh(output)
    # output = (output + 1) * 2 + 1
    return output.flatten()

# initial params
layers = [22, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 1]
key = random.PRNGKey(42)
params = init_params(layers, key)

# Loss Funciton
def mse_loss(params, x, y):
    preds = forward(params, x)
    return jnp.mean((preds - y) ** 2)

# L2 - Penalty optimizer
learning_rate = 0.001
optimizer = optax.adamw(learning_rate=learning_rate, weight_decay=1e-4)
opt_state = optimizer.init(params)

#
@jit
def update(params, opt_state, x_train, y_train, x_test, y_test):
    grads = grad(mse_loss)(params, x_train, y_train)
    grads = jax.tree_map(lambda g: jnp.clip(g, -1.0, 1.0), grads)  # gradient value clipping
    updates, opt_state = optimizer.update(grads, opt_state, params)
    params = optax.apply_updates(params, updates)
    train_preds = forward(params, x_train)
    train_loss = jnp.mean((train_preds - y_train) ** 2)
    test_preds = forward(params, x_test)
    test_loss = jnp.mean((test_preds - y_test) ** 2)
    return params, opt_state, train_loss, test_loss

In [ ]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# transforming to jnp array
X_train = jnp.array(X_train)
X_test = jnp.array(X_test)
y_train = jnp.array(y_train)
y_test = jnp.array(y_test)

best_params = None
best_loss = float('inf')
early_stopping_patience = 10
no_improvement_count = 0

# early stopping
for epoch in range(200):
    params, opt_state, train_loss, test_loss = update(params, opt_state, X_train, y_train, X_test, y_test)
    if test_loss < best_loss:
        best_loss = test_loss
        best_params = params
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/200], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

    if no_improvement_count >= early_stopping_patience:
        print("Early stopping due to no improvement.")
        break

We have 21 columns in our dataset. Because of that, our first layer which is input layer size is 22. Then we will test different numbers for hidden layers and neurons to find the optimal NN.

# For Submission

## RF model

In [ ]:
y_pred = rf_model.predict(test_df)
print(y_pred.tolist())
print(np.round(y_pred.tolist()).tolist())

[3.5169609390815557, 3.5218973396941355, 3.568136129275219, 4.344689183934351, 4.173398804160751, 3.860625234795918, 4.012933059497236, 4.1580438007519325, 3.873275878967688, 3.9037567686104877, 4.279393765955942, 3.979150245886739, 3.9205098684549347, 4.15123676681093, 4.255766733772671, 3.936699134795176, 3.626232096635432, 4.066722994353468, 4.022602935162937, 4.200644550012643, 4.1201465720858454, 3.856664354182312, 3.8678667649212164, 4.080128431807864, 3.8059142290490215, 3.957125133476539, 3.4053203339017113, 3.2432299514403424, 3.890534772954857, 3.469701165514971, 3.795335504792851, 3.694266834007342, 4.219969829418059, 3.6908532624922428, 3.833653917075803, 3.7362563479356807, 4.036680507686481, 3.3836560219291703, 3.321644904724333, 3.589084098151451, 3.398424797038707, 3.4658600564490376, 3.570179336530634, 3.426216129694885, 3.3270017502432423, 3.3546741449119035, 3.39123626177426, 3.3685070328698914, 3.32855667280457, 3.2475419394365495, 3.561306459015266, 3.5693952304056

In [ ]:
valid_steps = np.arange(0.5, 5.5, 0.5)  # Generates [0.5, 1.0, 1.5, ..., 5.0]



# Snap predictions to the nearest valid step
y_pred_snapped = np.array([valid_steps[np.argmin(np.abs(valid_steps - pred))] for pred in y_pred])
print(len(y_pred_snapped))
print(y_pred_snapped.tolist())

10000
[3.5, 3.5, 3.5, 4.5, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.5, 4.0, 4.0, 4.0, 4.5, 4.0, 3.5, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.5, 3.0, 4.0, 3.5, 4.0, 3.5, 4.0, 3.5, 4.0, 3.5, 4.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 4.0, 3.5, 3.5, 3.5, 3.0, 3.5, 3.5, 4.0, 4.0, 3.0, 3.0, 3.5, 4.0, 3.0, 3.5, 3.0, 3.5, 3.5, 3.5, 3.0, 3.0, 3.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 4.0, 3.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 4.0, 4.0, 3.0, 3.5, 3.5, 4.0, 4.0, 3.5, 3.5, 3.0, 3.0, 3.5, 3.5, 4.0, 3.5, 3.5, 3.5, 3.5, 4.0, 3.5, 3.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.0, 3.5, 3.5, 3.5, 3.5, 3.5, 4.0, 3.0, 4.0, 3.5, 3.0, 3.0, 3.0, 3.5, 3.0, 4.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.0, 3.5, 3.5, 3.5, 3.5, 4.0, 4.0, 4.0, 4.0, 3.5, 3.5, 4.0, 4.0, 4.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 4.0, 3.5, 3.5, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.5, 3.5, 3.5, 4.0, 4.0, 3.5, 4.0, 3.0, 4.0, 3.0, 3.5

## LR model

In [ ]:
log_pred = log_reg.predict(final_dataset)
print(log_pred.tolist())
log_pred = log_pred / 10.0
print(log_pred.tolist())

[40, 40, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 35, 35, 35, 35, 35, 35, 35, 35, 35, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 35, 35, 35, 35, 35, 35, 35, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 35, 40, 40, 40, 40, 40, 40, 40, 40, 35, 35, 35, 35, 35, 35, 35, 35, 35, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 35, 35, 35, 35, 35, 35, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 35, 35, 35, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 35, 35, 35, 35, 35, 35, 35, 35,

## NN model

In [ ]:
new_data = test_df.values
new_data = scaler.transform(new_data)


new_data = new_data.astype(jnp.float32)


predictions = forward(best_params, new_data)


print(len(predictions))
print(predictions.tolist())
print("Best Test Loss:", best_loss)



<ipython-input-16-ff2cf1fb3104>:49: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  grads = jax.tree_map(lambda g: jnp.clip(g, -1.0, 1.0), grads)  # 梯度剪裁


Epoch [10/200], Train Loss: 5.0770, Test Loss: 5.1483
Epoch [20/200], Train Loss: 1.4859, Test Loss: 1.4691
Epoch [30/200], Train Loss: 1.1986, Test Loss: 1.1943
Epoch [40/200], Train Loss: 1.0969, Test Loss: 1.0985
Epoch [50/200], Train Loss: 1.0608, Test Loss: 1.0602
Epoch [60/200], Train Loss: 1.0459, Test Loss: 1.0482
Epoch [70/200], Train Loss: 1.0371, Test Loss: 1.0402
Epoch [80/200], Train Loss: 1.0319, Test Loss: 1.0356
Epoch [90/200], Train Loss: 1.0283, Test Loss: 1.0332
Epoch [100/200], Train Loss: 1.0253, Test Loss: 1.0311
Epoch [110/200], Train Loss: 1.0227, Test Loss: 1.0293
Epoch [120/200], Train Loss: 1.0204, Test Loss: 1.0278
Epoch [130/200], Train Loss: 1.0182, Test Loss: 1.0264
Epoch [140/200], Train Loss: 1.0162, Test Loss: 1.0253
Epoch [150/200], Train Loss: 1.0143, Test Loss: 1.0242
Epoch [160/200], Train Loss: 1.0126, Test Loss: 1.0232
Epoch [170/200], Train Loss: 1.0109, Test Loss: 1.0223
Epoch [180/200], Train Loss: 1.0093, Test Loss: 1.0215
Epoch [190/200], Tr

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
